In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용

In [5]:
''' 파일 불러오기 '''
df = pd.read_csv("../../Sentiment_analysis/Primary_classification/Ternary_classification_2021.csv", encoding='UTF-8')
print(df.shape, df.columns)
df

(222289, 3) Index(['sentence', 'predicted_sentiment', 'org_idx'], dtype='object')


,sentence,predicted_sentiment,org_idx
0,세종시청 홈페이지 공공근로 알바 세종시 고용센터 일자리 지원센터 교차로 구인·구직 ...,긍정,0
1,최근에는 스마트시티 미래 차 모빌리티 생명 건강 스마트 그린 융합부품 소재산업 등 ...,긍정,0
2,사진 출처 세종시청 홈페이지 전반적으로 고용률은 높아지고 있지만 어르신 장애인 경력...,긍정,0
3,사진 출처 세종시 교차로 구인·구직 세종 재가 노인지원센터 세종시니어클럽 세종 재가...,긍정,0
4,세종시청 홈페이지 공공근로 등 공공일자리 사업 참여자 모집 공고 확인 및 참여 방법...,긍정,0
...,...,...,...
222284,이날 전달한 목제품은 편백나무로 만든 도마와 소나무로 만든 칼 꽂히다,부정,8431
222285,제작에 사용된 재료는 관내에서 생산한 목재를 활용했다,긍정,8431
222286,한편 시 관계자는 풍부한 산림자원을 활용해 실용성 높은 목제품을 만들고 어려운 이웃...,긍정,8431
222287,며 앞으로 더욱 다양한 목공활동을 통해 시민들이 목제품에 더욱 친근하게 다가설 수 ...,긍정,8431


In [6]:
for emo in ['긍정','부정','중립']:

    ''' 형태소 분석 '''
    nouns = []
    verbs = []
    adjs = []

    df_class = df[df['predicted_sentiment'] == "%s"%emo]
    corpus = df_class['sentence'].to_list()

    for sent in tqdm(range(len(corpus))):
        # 문장에서 형태소/품사 추출
        try:
            a = okt.pos(corpus[sent], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
            for x, y in a:
                # 품사가 명사면 명사 리스트에 단어 추가
                if y == 'Noun':
                    nouns.append(x)
                # 품사가 동사면 동사 리스트에 단어 추가
                elif y == 'Verb':
                    verbs.append(x)
                # 품사가 형용사면 형용사 리스트에 단어 추가
                elif y == 'Adjective':
                    adjs.append(x)
        except:
            pass

    morph_dic = {'명사':nouns, '동사':verbs, '형용사':adjs}
    for mor in ['명사','동사','형용사']:
        
        # 명사 리스트에서 명사 빈도 리스트 생성
        morph_cnt = Counter(morph_dic[mor]).most_common()

        morph = []
        morph_freq = []
        for key, val in morph_cnt:
            morph.append(key)
            morph_freq.append(val)

        # 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
        morph_df = pd.DataFrame({'morph':morph, 'morph_freq':morph_freq})
        morph_df.to_csv("2021_감정예측_%s_%s.csv"%(emo, mor), index=False, encoding='UTF-8')

100%|██████████| 188/188 [00:00<00:00, 208.56it/s]


In [7]:
import plotly.express as px

# unique 값들의 개수를 계산
sentiment_counts = df['predicted_sentiment'].value_counts()


# 막대그래프로 시각화
fig = px.bar(sentiment_counts, x=sentiment_counts.index, y=sentiment_counts.values,
             labels={'x': 'Sentiment', 'y': 'Count'},
             title='Sentiment Distribution 2021',
             color=sentiment_counts.index,
             text=sentiment_counts.values)  # 개수를 막대에 표시하기 위해 text 속성에 sentiment_counts.values 할당

# 개수를 막대 위에 표시하는 옵션 추가
fig.update_traces(texttemplate='%{text}', textposition='outside')

# y축 범위 설정 (기존 최대값의 10% 더 큰 값으로 설정)
y_axis_range = sentiment_counts.max() * 1.1
fig.update_layout(yaxis_range=[0, y_axis_range])

# y축 값의 형식 설정 (정수 형식으로 표시)
fig.update_layout(yaxis_tickformat=',d')

# 글꼴 크기 설정
fig.update_layout(
    font=dict(
        family="Arial, bold",
        size=13
    )
)

# x축 레이블 설정
fig.update_xaxes(title_text='Label')

# 제목을 가운데에 위치시키기
fig.update_layout(title_x=0.5)

# 그래프 출력
fig.show()